In [2]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import tiktoken

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
df = pd.read_csv("002_data/data_wide.csv", index_col=0)
df.head(3)

,seqn,gender,age,race,education,marital_status,pir,bmi,time1,time2,...,time2007,time2008,time2009,time2010,time2011,time2012,time2013,time2014,time2015,time2016
1,21009,1,55,3,3,1,3.79,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21010,2,52,3,4,6,1.24,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21012,1,63,4,3,6,0.89,0,1,0,...,1,1,0,0,0,1,1,0,0,0


In [6]:

time_columns = [col for col in df.columns if col.startswith('time')]

df[time_columns] = df[time_columns].astype(str)

def combine(row):
    combined_values = ' '.join(row[col] for col in time_columns)
    return combined_values

df['combined'] = df.apply(combine, axis=1)

embedding_encoding = "cl100k_base"
encoding = tiktoken.get_encoding(embedding_encoding)
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))

/var/folders/t3/mhnlg6pd5339qn7frt3kkty40000gn/T/ipykernel_38268/1362661259.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['combined'] = df.apply(combine, axis=1)
/var/folders/t3/mhnlg6pd5339qn7frt3kkty40000gn/T/ipykernel_38268/1362661259.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))


In [ ]:
df = df.drop(columns=[col for col in df.columns if col.startswith('time')])
df.to_csv("002_data/data_wide_embedding_initial.csv")

In [ ]:
df = pd.read_csv("002_data/data_wide_embedding_initial.csv", index_col=0)
df.head(2)

,seqn,gender,age,race,education,marital_status,pir,bmi,combined,n_tokens
1,21009,1,55,3,3,1,3.79,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,4031
2,21010,2,52,3,4,6,1.24,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,4031


In [ ]:
################### GPT ##############################

import tiktoken
from openai import OpenAI
import ast

# model: text-embedding-3-small
# default embedding dimension: 1536

from openai import OpenAI
api_key = ''
client = OpenAI(api_key=api_key)

def get_embedding(text, model="text-embedding-3-small", max_tokens=512):  # set the maximum equal to MOMENT
    text = text.replace("\n", " ")
    encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(text)[:max_tokens]
    text = encoding.decode(tokens)
    return client.embeddings.create(input=[text], model=model).data[0].embedding

embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 512   

df_gpt = df.copy() 
df_gpt["embedding"] = df_gpt.combined.apply(lambda x: get_embedding(x, model=embedding_model, max_tokens=max_tokens))
df_gpt["embedding"] = df_gpt["embedding"].apply(lambda x: str(x))

df_gpt = df_gpt[[col for col in df_gpt.columns if not col.startswith('n_tokens') and not col.startswith('time')]]
df_gpt.to_csv("009_untitled/trunc_wide_embedding_gpt1536.csv")
df_gpt.head(2)


,seqn,gender,age,race,education,marital_status,pir,bmi,combined,embedding
1,21009,1,55,3,3,1,3.79,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[-0.02994643896818161, 0.010527699254453182, 0..."
2,21010,2,52,3,4,6,1.24,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,"[-0.02028489299118519, 0.01061903778463602, 0...."
